# __Peer-graded Assignment Week 3__

## __Part 1 of 3__

### Task: __Create the initial dataframe taking the information from wikipedia__

In [1]:
#!pip install lxml  # a missing library

In [2]:
import pandas as pd

### Scrape the wikipedia page and convert it to dataframe

In [3]:
toronto = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = toronto[0]
df.head(3)
print('Initial shape:', df.shape)

Initial shape: (180, 3)


### Drop any row 'Not assigned' in borough

In [4]:
df = df[df.Borough!='Not assigned']
print('new shape:',df.shape)

new shape: (103, 3)


### Verify there is no Neighbourhood not assigned to any borough

In [5]:
df[df.Neighbourhood=='Not assigned']

,Postal Code,Borough,Neighbourhood


### Check df and shape

In [6]:
df.head(3)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
df.shape

(103, 3)

***

## __Part 2 of 3__

### Task: __Get the latitude and longitude coordinates of each neighborhood__

In [8]:
#!pip instal pgeocode
import pgeocode

### Get latitude and longitude from postal codes

In [9]:
lats = []
longs= []
# create nominatim object using iso country code of Canada (CA)
nomi = pgeocode.Nominatim('CA')
# get latitude and longitude from postal code
for postal_code in df['Postal Code']:
    lats.append(nomi.query_postal_code(postal_code).latitude)
    longs.append(nomi.query_postal_code(postal_code).longitude)
print('Length of latitudes, logitude lists:',len(lats),',',len(longs))

Length of latitudes, logitude lists: 103 , 103


### Add new data and clean df

In [10]:
df1 = df.copy()
df1['Latitude'] = lats
df1['Longitude'] = longs
# find any row with NaN info
df1[df1.isna().any(axis=1)]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
114,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


In [11]:
# drop NaN row
df1.dropna(inplace=True)
# reset index
df1.reset_index(drop=True,inplace=True)

In [12]:
df1.shape

(102, 5)

***

## __Part 3 of 3__

### Task: __Explore and cluster the neighborhoods in Toronto__

### Create map of Toronto and neighborhoods

In [13]:
import folium
from geopy.geocoders import Nominatim

In [14]:
# create map of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent='toronto')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)

In [15]:
# add the neighborhoods from df1
for lat, lng, borough, neighborhood in zip(df1['Latitude'],df1['Longitude'],df1['Borough'],df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],radius=5, popup=label, color='green', fill=True,parase_html=False).add_to(map_toronto)
map_toronto

### Get venues from Foursquare

In [16]:
# import new libraries
import json
import requests
import numpy as np
from pandas.io.json import json_normalize

In [17]:
# define some variables
CLIENT_ID = 'G35DEDCTV0G2QZHIJQYHJXWRJZDY1GAFWNFPWMMAFIEJ0L30' 
CLIENT_SECRET = 'N1KX1AWQCEQBWSYXMCJUTSB2JY204TO3F1O42BMKNKWRD0YJ'
VERSION = '20180605'
limit = 100
radius = 500

In [18]:
# Define a funtion to found venues to all neigbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
# Apply the function to Toronto and see how many venues were found
toronto_venues = getNearbyVenues(names=df1['Neighbourhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )
print('Total venues founded:',len(toronto_venues))

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

### Analyze each neigbourhood

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.tail()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store
2162,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2163,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2164,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2165,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2166,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
toronto_onehot.shape

(2167, 258)

In [22]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(3)

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# define a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
# create a new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Badminton Court,Skating Rink,Latin American Restaurant,Women's Store,Fast Food Restaurant,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant
1,"Alderwood, Long Branch",Convenience Store,Sandwich Place,Pizza Place,Coffee Shop,Pub,Gym,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
2,"Bathurst Manor, Wilson Heights, Downsview North",Mediterranean Restaurant,Middle Eastern Restaurant,Coffee Shop,Deli / Bodega,Other Repair Shop,Fried Chicken Joint,Pizza Place,Filipino Restaurant,Fish & Chips Shop,Fish Market
3,Bayview Village,Flower Shop,Golf Driving Range,Dog Run,Park,Gas Station,Trail,Falafel Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Pharmacy,Comfort Food Restaurant,Juice Bar,Sushi Restaurant,Breakfast Spot,Pizza Place


### Cluster neighborhoods

In [25]:
from sklearn.cluster import KMeans

In [26]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [27]:
# Create new dataframe that includes the cluster
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df1
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
# drop any nan value
toronto_merged.dropna(inplace=True)
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,2.0,Food & Drink Shop,Park,Women's Store,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
1,M4A,North York,Victoria Village,43.7276,-79.3148,1.0,Hockey Arena,French Restaurant,Coffee Shop,Intersection,Portuguese Restaurant,Park,Pizza Place,Fish & Chips Shop,Filipino Restaurant,Field
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,1.0,Coffee Shop,Breakfast Spot,Restaurant,Yoga Studio,Distribution Center,Food Truck,Spa,Event Space,Beer Store,Electronics Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,1.0,Clothing Store,Coffee Shop,Women's Store,Restaurant,Toy / Game Store,Sandwich Place,Cosmetics Shop,Sushi Restaurant,Men's Store,Bakery
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,1.0,Coffee Shop,Japanese Restaurant,Beer Bar,Burrito Place,Martial Arts School,Café,Mexican Restaurant,Ethiopian Restaurant,Ramen Restaurant,Bubble Tea Shop


### Visualize the clusters

In [28]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [29]:
# validate columns 'Cluster labels' is type 'int'
toronto_merged = toronto_merged.astype({'Cluster Labels':int})

In [30]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [31]:
# Let's see the size of every cluster
toronto_merged['Cluster Labels'].value_counts()

1    84
2    12
0     2
4     1
3     1
Name: Cluster Labels, dtype: int64

#### We can see on 2nd cluster the most visited venue are Parks

In [32]:
# review the second cluster creating a new dataframe
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,:]
# on this cluster we can validate that 1st, 2nd & 3rd Most common venue
print('cluster2 common venues\n',cluster2['1st Most Common Venue'].value_counts().head(3))
print('cluster2 common venues\n',cluster2['2nd Most Common Venue'].value_counts().head(3))
print('cluster2 common venues\n',cluster2['3rd Most Common Venue'].value_counts().head(3))

cluster2 common venues
 Park                   5
Sporting Goods Shop    1
Photography Studio     1
Name: 1st Most Common Venue, dtype: int64
cluster2 common venues
 Park                                        5
Home Service                                2
Residential Building (Apartment / Condo)    1
Name: 2nd Most Common Venue, dtype: int64
cluster2 common venues
 Gym              2
Park             2
Women's Store    2
Name: 3rd Most Common Venue, dtype: int64


#### We can see on 1st cluster the most visited venue are Coffe shop or Café

In [33]:
# review the first cluster creating a new dataframe
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,:]
# on this cluster we can validate that 1st, 2nd & 3rd Most common venue
print('cluster1 common venues\n',cluster1['1st Most Common Venue'].value_counts().head(3))
print('cluster1 common venues\n',cluster1['2nd Most Common Venue'].value_counts().head(3))
print('cluster1 common venues\n',cluster1['3rd Most Common Venue'].value_counts().head(3))

cluster1 common venues
 Coffee Shop    17
Pizza Place    10
Café            5
Name: 1st Most Common Venue, dtype: int64
cluster1 common venues
 Coffee Shop      12
Café              7
Women's Store     3
Name: 2nd Most Common Venue, dtype: int64
cluster1 common venues
 Coffee Shop    8
Pizza Place    5
Restaurant     5
Name: 3rd Most Common Venue, dtype: int64
